In [1]:
import numpy as np
import pandas as pd

%cd ..
from src.inference_testing import *
%cd -

/home/ec2-user/SageMaker/data-science-product-image


Using TensorFlow backend.


/home/ec2-user/SageMaker/data-science-product-image/notebooks


In [2]:
taxonomy_s3_path = 's3://data-science-training-datasets/product-image/mturk-50k-v2/taxonomy.json'

In [4]:
load_from_s3(taxonomy_s3_path, '../files/taxonomy.json')
taxonomy = load_taxonomy('../files/taxonomy.json')
H = Hierarchy(taxonomy, children_key='subcategories')
    
taxonomy_df = pd.DataFrame(H.taxonomy)
taxonomy_df['parent_name'] = taxonomy_df['parent'].apply(lambda x: str(taxonomy_df[taxonomy_df['id']==x]['name'].values)).apply(lambda x: x.replace('[','').replace(']','').replace("'",'').replace("'",''))
taxonomy_df['grandparent_id'] = taxonomy_df['parent'].apply(lambda x: str(taxonomy_df[taxonomy_df['id']==x]['parent'].values)).apply(lambda x: x.replace('[','').replace(']','').replace('.',''))
taxonomy_df['grandparent_name'] = taxonomy_df['grandparent_id'].apply(lambda x: str(taxonomy_df[taxonomy_df['id'].astype(str)==x]['name'].values)).apply(lambda x: x.replace('[','').replace(']','').replace("'",'').replace("'",''))

taxonomy_df['category1'] = np.where(np.logical_and(taxonomy_df["grandparent_name"]!='',taxonomy_df["grandparent_name"]!='Main')
                                        , taxonomy_df["grandparent_name"] 
                                        , np.where(
                                            np.logical_and(taxonomy_df["parent_name"]!='',taxonomy_df["parent_name"]!='Main'), 
                                            taxonomy_df["parent_name"],
                                            taxonomy_df["name"]))
taxonomy_df['category2'] = np.where(np.logical_or(taxonomy_df["grandparent_name"]=='',taxonomy_df["grandparent_name"]=='Main')
                                        , taxonomy_df["name"]
                                        , taxonomy_df["parent_name"])
taxonomy_df['category3'] = np.where(taxonomy_df["category2"]!=taxonomy_df['name']
                                        , taxonomy_df["name"]
                                        , '')
taxonomy_df['full_category'] = np.where(taxonomy_df['category3']!='',
                                           taxonomy_df['category1']+'->'+taxonomy_df['category2']+'->'+taxonomy_df['category3'],
                                           np.where(taxonomy_df['category2']!='',
                                                   taxonomy_df['category1']+'->'+taxonomy_df['category2'],
                                                   taxonomy_df['category1']))
taxonomy_df = taxonomy_df[['id', 'category1', 'category2','category3','full_category']]

taxonomy_df

,id,category1,category2,category3,full_category
0,0,Clothing,Pants,Jeans,Clothing->Pants->Jeans
1,1,Clothing,Dresses,,Clothing->Dresses
2,2,Clothing,Intimate Wear,,Clothing->Intimate Wear
3,3,Clothing,Coats & Jackets,,Clothing->Coats & Jackets
4,4,Clothing,Jumpers & Rompers,,Clothing->Jumpers & Rompers
...,...,...,...,...,...
63,63,Accessories,Jewelry,,Accessories->Jewelry
64,64,Home & Decor,Home & Decor,,Home & Decor->Home & Decor
65,65,Home & Decor,Decor,,Home & Decor->Decor
66,66,Home & Decor,Furniture,,Home & Decor->Furniture


In [7]:
testing_df=pd.read_json('../files/testing.json')
testing_df

,product_id,target_image_url,title,description,confidence,original_label,human_group,category_id,category_name,one_hot_list,human_group_id,human_group_one_hot_list,image
35104,144723440,s3://data-science-training-datasets/product-im...,"Schott Zwiesel Classico Martini Glass, Set of 6",Designed to convey the flavor profile and char...,0.666667,77,None,48,Kitchen,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[1.0, 0.0, 0.0, 0.0, 0.0]",144723440.jpg
40068,146574520,s3://data-science-training-datasets/product-im...,Ruffle Sleeve Crepe Jumpsuit,Free shipping and returns on Eliza J Ruffle Sl...,1.000000,10,Women,4,Jumpers & Rompers,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[0.0, 1.0, 0.0, 0.0, 0.0]",146574520.jpg
17541,134158571,s3://data-science-training-datasets/product-im...,"Look At Me Now Seamless Leggings, Mini Leopard",<ul> <li>The Slim is Built In™— double-layer w...,1.000000,11,Women,5,Leggings,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...",1,"[0.0, 1.0, 0.0, 0.0, 0.0]",134158571.jpg
19291,136114623,s3://data-science-training-datasets/product-im...,Ingenuity™ SmartBounce Automatic Bouncer™ in B...,The Ingenuity SmartBounce Automatic Bouncer so...,1.000000,67,Baby,41,Baby Gear,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",4,"[0.0, 0.0, 0.0, 0.0, 1.0]",136114623.jpg
47858,151352481,s3://data-science-training-datasets/product-im...,Little Tikes TotSports Easy Score Toy Basketba...,Free 2-day shipping on qualified orders over $...,0.666667,68,Kids,42,Toys & Games,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",3,"[0.0, 0.0, 0.0, 1.0, 0.0]",151352481.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...
39028,146170024,s3://data-science-training-datasets/product-im...,Women's Catlin Double Gore Ankle Bootie - Univ...,Free shipping on orders of $35+ from Target. R...,0.666667,31,Women,15,Boots,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[0.0, 1.0, 0.0, 0.0, 0.0]",146170024.jpg
50985,157892080,s3://data-science-training-datasets/product-im...,Insignia™ 3.4qt Digital Air Fryer Black NS-AF3...,Shop Insignia™ 3.4qt Digital Air Fryer Black a...,1.000000,77,None,48,Kitchen,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[1.0, 0.0, 0.0, 0.0, 0.0]",157892080.jpg
23862,139515697,s3://data-science-training-datasets/product-im...,Good Vibes Faux Leather Moto Jacket,Free shipping and returns on BLANKNYC Good Vib...,0.666667,9,Women,3,Coats & Jackets,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[0.0, 1.0, 0.0, 0.0, 0.0]",139515697.jpg
48880,152897377,s3://data-science-training-datasets/product-im...,Safari | KiwiCo,Introduce Zoology to your little one! Everyday...,1.000000,68,Kids,42,Toys & Games,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",3,"[0.0, 0.0, 0.0, 1.0, 0.0]",152897377.jpg


In [11]:
outputs=np.load('../files/pred-post-finetune.npy', allow_pickle=True)

In [12]:
y_pred=np.argmax(outputs.item().get('probabilities'), axis=1)

In [55]:
columns=['True Root Category', 'Predicted Root Category', 'True Child Category', 'Predicted Child Category', 'True Grandchild Category', 'Predicted Grandchild Category']

comparison_df=pd.DataFrame(columns=columns)

In [56]:
testing_df=testing_df[['category_id', 'category_name']]
testing_df

,category_id,category_name
35104,48,Kitchen
40068,4,Jumpers & Rompers
17541,5,Leggings
19291,41,Baby Gear
47858,42,Toys & Games
...,...,...
39028,15,Boots
50985,48,Kitchen
23862,3,Coats & Jackets
48880,42,Toys & Games


In [57]:
testing_hierarchy_df=pd.merge(testing_df, taxonomy_df, how='left', left_on='category_id',right_on='id')
testing_hierarchy_df.drop(['category_id', 'category_name'], axis=1)

,id,category1,category2,category3,full_category
0,48,Home & Decor,Kitchen,,Home & Decor->Kitchen
1,4,Clothing,Jumpers & Rompers,,Clothing->Jumpers & Rompers
2,5,Clothing,Pants,Leggings,Clothing->Pants->Leggings
3,41,Baby Gear,Baby Gear,,Baby Gear->Baby Gear
4,42,Toys & Games,Toys & Games,,Toys & Games->Toys & Games
...,...,...,...,...,...
5165,15,Shoes,Boots,,Shoes->Boots
5166,48,Home & Decor,Kitchen,,Home & Decor->Kitchen
5167,3,Clothing,Coats & Jackets,,Clothing->Coats & Jackets
5168,42,Toys & Games,Toys & Games,,Toys & Games->Toys & Games


In [58]:
comparison_df['True Root Category']=testing_hierarchy_df['category1']
comparison_df.head()

,True Root Category,Predicted Root Category,True Child Category,Predicted Child Category,True Grandchild Category,Predicted Grandchild Category
0,Home & Decor,NaN,NaN,NaN,NaN,NaN
1,Clothing,NaN,NaN,NaN,NaN,NaN
2,Clothing,NaN,NaN,NaN,NaN,NaN
3,Baby Gear,NaN,NaN,NaN,NaN,NaN
4,Toys & Games,NaN,NaN,NaN,NaN,NaN


In [59]:
for i in range(len(testing_df)):
    comparison_df['Predicted Root Category'][i]=taxonomy_df['category1'][y_pred[i]]

In [60]:
comparison_df.head()

,True Root Category,Predicted Root Category,True Child Category,Predicted Child Category,True Grandchild Category,Predicted Grandchild Category
0,Home & Decor,Home & Decor,NaN,NaN,NaN,NaN
1,Clothing,Clothing,NaN,NaN,NaN,NaN
2,Clothing,Clothing,NaN,NaN,NaN,NaN
3,Baby Gear,Baby Gear,NaN,NaN,NaN,NaN
4,Toys & Games,Toys & Games,NaN,NaN,NaN,NaN


In [61]:
correct_count=0

for i in range(len(comparison_df)):
    if comparison_df['True Root Category'][i]==comparison_df['Predicted Root Category'][i]:
        correct_count +=1

print(correct_count/len(comparison_df))

0.969825918762089


In [62]:
comparison_df['True Child Category']=testing_hierarchy_df['category2']

In [63]:
for i in range(len(testing_df)):
    comparison_df['Predicted Child Category'][i]=taxonomy_df['category2'][y_pred[i]]

In [64]:
correct_count=0

for i in range(len(comparison_df)):
    if comparison_df['True Child Category'][i]==comparison_df['Predicted Child Category'][i]:
        correct_count +=1

print(correct_count/len(comparison_df))

0.927852998065764


In [65]:
taxonomy_df.category3.unique()

array(['Jeans', '', 'Leggings', 'Activewear', 'Cardigans', 'Bracelets',
       'Earrings', 'Necklaces', 'Rings', 'Watches', 'Vests', 'Rugs',
       'Indoor Furniture', 'Outdoor Furniture'], dtype=object)

In [66]:
comparison_df['True Grandchild Category']=testing_hierarchy_df['category3']

In [69]:
for i in range(len(testing_df)):
    comparison_df['Predicted Grandchild Category'][i]=taxonomy_df['category3'][y_pred[i]]

In [70]:
comparison_df

,True Root Category,Predicted Root Category,True Child Category,Predicted Child Category,True Grandchild Category,Predicted Grandchild Category
0,Home & Decor,Home & Decor,Kitchen,Kitchen,,
1,Clothing,Clothing,Jumpers & Rompers,Jumpers & Rompers,,
2,Clothing,Clothing,Pants,Pants,Leggings,Leggings
3,Baby Gear,Baby Gear,Baby Gear,Baby Gear,,
4,Toys & Games,Toys & Games,Toys & Games,Toys & Games,,
...,...,...,...,...,...,...
5165,Shoes,Shoes,Boots,Boots,,
5166,Home & Decor,Electronics,Kitchen,Electronics,,
5167,Clothing,Clothing,Coats & Jackets,Coats & Jackets,,
5168,Toys & Games,Toys & Games,Toys & Games,Toys & Games,,


In [74]:
grandchild_count=0
correct_count=0

for i in range(len(comparison_df)):
    if comparison_df['True Grandchild Category'][i]!='':
        grandchild_count +=1
        if comparison_df['True Grandchild Category'][i]==comparison_df['Predicted Grandchild Category'][i]:
            correct_count +=1

print(correct_count/grandchild_count)
grandchild_count

0.9354144241119483


929